# Interoperability with C

Julia has a simple way to call C and Fortran functions in shared libraries, via the `ccall` function.

In [ ]:
?ccall

We see that we must specify:

- the name of the function, as a Julia symbol or as a string
- and the name of the shared library where it lives;  these are given as an ordered pair (tuple)


- the return type of the function

- the argument types that the function accepts, as a tuple

- and the arguments themselves

A simple example is to call the clock function:

In [ ]:
t = ccall( (:clock, "libc"), Int32, ())

In [ ]:
typeof(t)

In [ ]:
path = ccall( (:getenv, "libc"), Ptr{UInt8}, (Ptr{UInt8},), "PATH")

Here, `Ptr` denotes a pointer to the given type.

In [ ]:
unsafe_string(path) # ENV["PATH"]

In [ ]:
?unsafe_string

C libraries tend to have large, repetitive, uniform API patterns and naming conventions.

Julia macros combined with `ccall` can make it easy to write wrapper interfaces.

Example: MPFR for `BigFloat`:

In [ ]:
a = big"0.1"
b = big"0.3"
@which a + b

In [ ]:
@which sin(a)

Automated wrapper generation from headers is possible via the Clang.jl package: https://github.com/ihnorton/Clang.jl

# Interoperability with Python: the `PyCall` package

Coming from the Python ecosystem, the range of packages available in Julia can seem somewhat limited.
This is offset, however, by the ease of calling out to packages written in other languages from within Julia.

In particular, Python interoperability is very easy, thanks to the [`PyCall` package](https://github.com/stevengj/PyCall.jl). PyCall uses `ccall` to execute the same operations and C API from `libpython` that the Python interpreter would use.

In [ ]:
using PyCall

`PyCall` has a high-level interface that is designed so that the "transport" between Julia and Python is transparent from the user's point of view. For example, to import the Python `math` module, we do

In [ ]:
@pyimport math

We can now mix and match Python calls, labelled by the `math.` qualifier, and Julia calls:

In [ ]:
math.sin(0.3*math.pi) - sin(0.3*pi) 

Array objects are automatically converted:

In [ ]:
@pyimport numpy.random as nprandom
nprandom.rand(3,4)

Let's define a Julia function:

In [ ]:
objective = x -> cos(x) - x

This is the Julia syntax for an anonymous function (like `lambda` in Python).

In [ ]:
objective(3)

We can pass this Julia function to a Python module:

In [ ]:
@pyimport scipy.optimize as so
so.newton(objective, 1)

In [ ]:
objective(ans)

The main difference from Python is how to access member elements of Python structures.

Julia has ODE solvers in the `ODE.jl` and `Sundials.jl` packages. But we can also call Python solvers:

In [ ]:
@pyimport scipy.integrate as integrate

In [ ]:
f(x,t) = -x  

In [ ]:
t = colon(0,0.1,10)

In [ ]:
soln = integrate.odeint(f, 1, linspace(0,10,101))

In [ ]:
soln = integrate.odeint(f, 1, t)

In [ ]:
using PyPlot

The `PyPlot` package uses `PyCall` to provide a Julia wrapper API around `matplotlib`.

In [ ]:
plot(t, soln)

In [ ]:
plot(t, exp(-t))

## Subtle difference from Python

Accessing fields (properties) and methods of Python objects uses the `obj.a` and `obj.b()` syntax, where `obj` is a Python object.
However, currently the `obj.b` syntax in Julia is restricted to accessing fields of Julia composite types.

For this reason, to access fields and methods of Python objects via PyCall, it is necessary to use the syntax

`obj[:a]` for fields, and

`obj[:a]()` for methods

Here, we are using the Julia syntax `:a` to mean the Julia symbol `a`.

## Lower level

The high-level `PyCall` interface is built on top of a lower-level interface which deals with the "transport" of objects between Python and Julia, based on a `PyObject` Julia type that wraps `PyObject*` in C, and represents a reference to a Python object.



In [ ]:
PyObject(3)

In [ ]:
x = rand(5, 5)

In [ ]:
px = PyObject(x)

In [ ]:
typeof(px)

In [ ]:
fieldnames(px)

In [ ]:
px.o

In [ ]:
# px.shape  in Python becomes:
px[:shape]

In [ ]:
typeof(ans)  # the result has already been translated back into a Julia object

Julia arrays are passed into Python without a copy. By default the resulting Python array is copied when a result is requested in Julia; this can be avoided at a lower level using `pycall` and `PyArray`.

**Exercise**: Use your favorite Python package from Julia!